In [1]:
!unzip '/content/drive/MyDrive/ColabNotebooks/IRTG/Encode_the_Qode/Encode-the-Qode/data/raw/ArXiv/archive (1).zip'

Archive:  /content/drive/MyDrive/ColabNotebooks/IRTG/Encode_the_Qode/Encode-the-Qode/data/raw/ArXiv/archive (1).zip
  inflating: arxiv_data.csv          
  inflating: arxiv_data_210930-054931.csv  


## LOAD TOKENS

In [44]:
import pandas as pd
from sklearn.model_selection import train_test_split

import json
import ast
import pickle

import nltk
nltk.download('stopwords')

from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
RS = 42
DATA_PATH = '/content/drive/MyDrive/ColabNotebooks/IRTG/Encode_the_Qode/Encode-the-Qode/data'

In [37]:
data = pd.read_csv('arxiv_data_210930-054931.csv')
q_data = pd.read_csv(DATA_PATH +'/preprocessed/Quantinar/course_chapters_202309061748.csv', delimiter=';')

In [ ]:
tokens_arxiv = []

data['titles'].apply(lambda title: tokens_arxiv.extend(title.lower().split()))
data['abstracts'].apply(lambda title: tokens_arxiv.extend(title.lower().split()))

In [39]:
tokens_quantinar = []
q_data.loc[q_data["'word_cloud'"].notna(),"'word_cloud'"].apply(lambda x: tokens_quantinar.extend(list(ast.literal_eval(x).keys())))
q_data.loc[(q_data["'chapter_name'"].notna()) & (q_data["'chapter_name'"].apply(lambda x: isinstance(x, dict))),"'chapter_name'"].apply(lambda x: tokens_quantinar.extend(list(ast.literal_eval(x).values())))

Series([], Name: 'chapter_name', dtype: object)

In [40]:
ps = PorterStemmer()

In [41]:
tokens_arxiv = [ps.stem(token) for token in set(tokens_arxiv) if token not in stopwords.words('english')]
tokens_quantinar = [ps.stem(token.lower()) for token in set(tokens_quantinar) if token.lower() not in stopwords.words('english')]

In [46]:
with open(DATA_PATH +'/preprocessed/Quantinar/text_tokens.pkl', 'wb') as handle:
    pickle.dump(tokens_quantinar, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(DATA_PATH +'/preprocessed/ArXiv/text_tokens.pkl', 'wb') as handle:
    pickle.dump(tokens_arxiv, handle, protocol=pickle.HIGHEST_PROTOCOL)

## COMPUTE OVERLAP

In [50]:
QPATH = "Quantlet/7-corpus-token-ident"

In [51]:
import sys
IN_COLAB = 'google.colab' in sys.modules

import os
if IN_COLAB:
  os.chdir(f'/content/drive/MyDrive/ColabNotebooks/IRTG/Encode_the_Qode/Encode-the-Qode/{QPATH}')

sys.path.append('../src')



In [54]:
%pip install transformers[torch]
%pip install -q sentencepiece
%pip install datasets==2.13.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.0 MB/s eta 0:00:00


In [55]:
from datasets import load_dataset

In [57]:
train_dataset = load_dataset("json",
                             data_files="train_dataset_descr.json",
                             field="data",
                             data_dir="../../data/preprocessed/Quantlet/")
test_dataset = load_dataset("json",
                            data_files="test_dataset_descr.json",
                            field="data",
                            data_dir="../../data/preprocessed/Quantlet/")

test_dataset = load_dataset("json",
                            data_files="val_dataset_json.json",
                            field="data",
                            data_dir="../../data/preprocessed/Quantlet/")

  0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-3f18bb67f140614e/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-9fd014b5cbf2c454/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [63]:
train_dataset['train']['output_sequence']

['Applies the adaptive algorithm on the model proposed by Barrow (2006) to capture the disaster of stock market and forecast stock price.',
 'Scenario analysis for an ETF on the CRIX (Haerdle and Trimborn 2015). Requires Cryptocurrency Data from https://box.hu-berlin.de/d/588d5e7e5de84ae58a51/ ',
 'Scenario analysis for an ETF on the CRIX (Haerdle and Trimborn 2015). Requires Cryptocurrency Data from https://box.hu-berlin.de/d/588d5e7e5de84ae58a51/ ',
 'Scenario analysis for an ETF on the CRIX (Haerdle and Trimborn 2015). Requires Cryptocurrency Data from https://box.hu-berlin.de/d/588d5e7e5de84ae58a51/ ',
 'Scenario analysis for an ETF on the CRIX (Haerdle and Trimborn 2015). Requires Cryptocurrency Data from https://box.hu-berlin.de/d/588d5e7e5de84ae58a51/ ',
 'Scenario analysis for an ETF on the CRIX (Haerdle and Trimborn 2015). Requires Cryptocurrency Data from https://box.hu-berlin.de/d/588d5e7e5de84ae58a51/ ',
 'Data visualisation for phacking talk',
 'Data visualisation for phac